# This Notebook is my try to detect labels according to SSVEP Signals

In [30]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import os
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

In [31]:
!ls /kaggle/input/mtcaic3
base_path = "/kaggle/input/mtcaic3/"

MI	   sample_submission.csv  test.csv   validation.csv
README.md  SSVEP		  train.csv


## Here I load the data as pandas dataframe to analyze the data and to get the correlation between different features

In [32]:
def load_trial_data(row, base_path='.'):
    id_num = row['id']
    if id_num <= 4800:
        dataset = 'train'
    elif id_num <= 4900:
        dataset = 'validation'
    else:
        dataset = 'test'

    eeg_path = f"{base_path}/{row['task']}/{dataset}/{row['subject_id']}/{row['trial_session']}/EEGdata.csv"
    eeg_data = pd.read_csv(eeg_path)

    trial_num = int(row['trial'])
    samples_per_trial = 1750 if row['task'] == 'SSVEP' else 2250
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial - 1
    return eeg_data.iloc[start_idx:end_idx + 1]

def load_all_trials_df(df, base_path, start, end):
    trials = []
    for i in range(start, end):
        row = df.iloc[i]
        trial_df = load_trial_data(row, base_path)
        trials.append(trial_df)

    return trials


In [33]:
train_df = pd.read_csv(os.path.join(base_path, 'train.csv'))
validation_df = pd.read_csv(os.path.join(base_path, 'validation.csv'))
test_df = pd.read_csv(os.path.join(base_path, 'test.csv'))

In [66]:
train_x = load_all_trials_df(train_df, base_path,2400,4800)
val_x = load_all_trials_df(validation_df, base_path,50,100)
test_x = load_all_trials_df(test_df, base_path,50,100)

In [67]:


# Assume train_x is a list of DataFrames
num_samples = len(train_x)
columns = train_x[0].columns

# Initialize correlation table with zeros
correlation_table = pd.DataFrame(0.0, index=columns, columns=columns)

# Accumulate correlations
for df in train_x:
    for col1 in columns:
        for col2 in columns:
            corr = df[col1].corr(df[col2])
            if pd.notnull(corr):
                correlation_table.loc[col1, col2] += corr

# Average over all samples
correlation_table /= num_samples

# Round and print
print(correlation_table.round(3))


/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


             Time     FZ     C3     CZ     C4     PZ    PO7     OZ    PO8  \
Time        1.000  0.011 -0.021 -0.050 -0.080 -0.025  0.028 -0.014 -0.008   
FZ          0.011  1.000  0.464  0.588  0.550  0.372  0.202  0.234  0.253   
C3         -0.021  0.464  1.000  0.739  0.610  0.665  0.757  0.713  0.672   
CZ         -0.050  0.588  0.739  1.000  0.867  0.812  0.640  0.715  0.731   
C4         -0.080  0.550  0.610  0.867  1.000  0.782  0.590  0.680  0.741   
PZ         -0.025  0.372  0.665  0.812  0.782  1.000  0.742  0.845  0.869   
PO7         0.028  0.202  0.757  0.640  0.590  0.742  1.000  0.824  0.811   
OZ         -0.014  0.234  0.713  0.715  0.680  0.845  0.824  1.000  0.890   
PO8        -0.008  0.253  0.672  0.731  0.741  0.869  0.811  0.890  1.000   
AccX       -0.000 -0.000 -0.001 -0.000 -0.002  0.002  0.004  0.002  0.001   
AccY        0.038  0.001 -0.002 -0.003 -0.003  0.004  0.004  0.001 -0.000   
AccZ        0.062  0.003 -0.006 -0.002 -0.003  0.003  0.010  0.002  0.002   

In [68]:
# --- 3. Your data loading functions ---

def load_trial_data(row, base_path='.'):
    id_num = row['id']
    if id_num <= 4800:
        dataset = 'train'
    elif id_num <= 4900:
        dataset = 'validation'
    else:
        dataset = 'test'

    eeg_path = f"{base_path}/{row['task']}/{dataset}/{row['subject_id']}/{row['trial_session']}/EEGdata.csv"
    eeg_data = pd.read_csv(eeg_path)

    trial_num = int(row['trial'])
    samples_per_trial = 1750 if row['task'] == 'SSVEP' else 2250
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial - 1
    return eeg_data.iloc[start_idx:end_idx + 1]

def load_all_trials(df, base_path, channels, start, end):
    trials = []
    for i in range(start, end):
        row = df.iloc[i]
        trial_df = load_trial_data(row, base_path)[channels].to_numpy()
        trials.append(trial_df)

    trials = np.stack(trials)
    return trials


### The Chosen features according to the correlation table

In [69]:
ssvep_channels = ['PO8', 'C4', 'FZ','C3','Time','AccX' ,  'AccY' ,  'AccZ' , 'Gyro1'  ,'Gyro2'  ,'Gyro3']

### Loading the data as numpy arrays

In [70]:
train_x = load_all_trials(train_df, base_path, ssvep_channels,2400,4800)
val_x = load_all_trials(validation_df, base_path, ssvep_channels,50,100)
test_x = load_all_trials(test_df, base_path, ssvep_channels,50,100)

# Since load_all_trials now returns a list, you can check the number of trials loaded
print("Number of trials in train_x:", len(train_x))
print("Number of trials in val_x:", len(val_x))
print("Number of trials in test_x:", len(test_x))

# You can also inspect the shape of the first few trials to see the different lengths
print("Shape of the first trial in train_x:", train_x[0].shape)
print("Shape of the first trial in val_x:", val_x[0].shape)
print("Shape of the first trial in test_x:", test_x[0].shape)

Number of trials in train_x: 2400
Number of trials in val_x: 50
Number of trials in test_x: 50
Shape of the first trial in train_x: (1750, 11)
Shape of the first trial in val_x: (1750, 11)
Shape of the first trial in test_x: (1750, 11)


In [71]:
train_labels=pd.read_csv(base_path+"train.csv")
train_labels = train_labels[0:2400]

val_labels=pd.read_csv(base_path+"validation.csv")
val_labels = val_labels[0:50]

In [72]:
train_labels = train_labels.iloc[:,-1]
val_labels = val_labels.iloc[:,-1]

In [73]:
label_to_index = {
    "Left": 0,
    "Right": 1,
    "Forward":2,
    "Backward":3
}
index_to_label = {
    0: "Left",
    1: "Right",
    2: "Forward",
    3: "Backward"
}
train_labels = [label_to_index[label] for label in train_labels]
val_labels = [label_to_index[label] for label in val_labels]


In [74]:
train_labels = torch.tensor(train_labels, dtype=torch.float32).unsqueeze(1)  # shape: [N, 1]
val_labels = torch.tensor(val_labels, dtype=torch.float32).unsqueeze(1)

In [75]:
train_x = torch.from_numpy(train_x)
val_x = torch.from_numpy(val_x)

In [76]:
train_x.shape

torch.Size([2400, 1750, 11])

### Constructing the model

In [44]:
class TransformerClassifier(nn.Module):
    def __init__(self, input_dim=11, d_model=32, num_heads=4, num_layers=3, num_classes=4, dropout=0.1):
        super(TransformerClassifier, self).__init__()

        self.input_bn = nn.BatchNorm1d(input_dim)

        # Linear projection of input features to d_model
        self.input_proj = nn.Linear(input_dim, d_model)

        # Positional encoding: learnable or sinusoidal
        self.pos_encoding = nn.Parameter(torch.randn(1, 1750, d_model))  # Assuming max length 1750

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.output_bn = nn.BatchNorm1d(d_model)
        self.fc = nn.Linear(d_model, 4)

    def forward(self, x):
        # x: [B, T, F] → BatchNorm
        x = self.input_bn(x.permute(0, 2, 1)).permute(0, 2, 1)  # [B, T, F]
    
        x = self.input_proj(x)  # [B, T, d_model]
        x = x + self.pos_encoding[:, :x.size(1), :]  # Add positional encoding
    
        out = self.transformer(x)  # [B, T, d_model]
        pooled = out[:, -1, :]  # Use last token's representation
    
        pooled = self.output_bn(pooled)  # BN before final layer
        logits = self.fc(pooled)  # [B, 1]
        return logits


In [36]:
class SequenceDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Return input sequence and label
        return self.data[idx], self.labels[idx]


train_dataset = SequenceDataset(train_x, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = SequenceDataset(val_x, val_labels)
val_loader=DataLoader(val_dataset, batch_size=32, shuffle=True)

In [55]:
model = TransformerClassifier()

In [52]:

def evaluate(model, dataloader, device='cuda'):
    model.eval()
    correct = total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x_batch, y_batch in dataloader:
            x_batch = x_batch.to(device, torch.float32)
            y_batch = y_batch.to(device).long().view(-1)  # Integer class labels

            logits = model(x_batch)              # Shape: [B, num_classes]
            preds = torch.argmax(logits, dim=1)  # Shape: [B]

            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())

    accuracy = 100.0 * correct / total
    f1 = f1_score(all_labels, all_preds, average='weighted')  # or 'macro'/'micro' if needed

    return accuracy, f1


In [53]:
max_acc=0

In [ ]:

def train(model, train_loader, val_loader, num_epochs=10, lr=1e-3, device='cuda'):
    global max_acc, max_model
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for x_batch, y_batch in train_loader:
            x_batch = x_batch.to(device, torch.float32)
            y_batch = y_batch.to(device).long().view(-1)  # e.g., [32]

              # BCE expects float [B, 1]

            optimizer.zero_grad()
            logits = model(x_batch)  # [B, 1]
            loss = criterion(logits, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)

        # Evaluate on train and val
        #train_acc, train_f1 = evaluate(model, train_loader, device)
        val_acc, val_f1 = evaluate(model, val_loader, device)
        if val_acc > max_acc:
            max_acc = val_acc
            torch.save(model.state_dict(), "best_model_ssvep.pt")

        print(
            f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f} | "
            #f"Train Acc: {train_acc:.2f}% - F1: {train_f1:.2f} | "
            f"Val Acc: {val_acc:.2f}% - F1: {val_f1:.2f}"
        )

for _ in range(10):
    model = TransformerClassifier()
    train(model,train_loader,val_loader,35)

print(max_acc)

In [64]:
model_best = TransformerClassifier()
model_best.load_state_dict(torch.load("/kaggle/working/best_model_ssvep.pt", map_location="cpu"))
model_best.to("cpu")
model_best.eval()  # optional but recommended during inference

test = torch.from_numpy(test_x).to(torch.float32)  # ensure float32
test = test.to("cpu")  # ensure it's also on CPU

logits = model_best(test)


### In this try I succesfully got 66 accuracy for a classification for 4 classes

In [65]:
model_best.to("cuda")
evaluate(model_best, val_loader, "cuda")

(66.0, 0.6590289677682578)